In [ ]:
# Import necessary modules
from loguru import logger
from datetime import datetime
from src.elt.transforms.utils import connect_mongodb

2025-08-17 00:20:43.323 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: /Users/Edward/Documents/GitHub/book-club


In [2]:
def count_books_read_this_year():
    """
    Connects to the MongoDB database and counts the number of books read
    by all users in the current year.
    """
    db, client = connect_mongodb()

    # Get the current year
    current_year = datetime.now().year

    # Define the aggregation pipeline
    pipeline = [
        # Match documents with a completed date in the current year
        {
            "$match": {
                "date_completed": {
                    "$gte": datetime(current_year, 1, 1),
                    "$lt": datetime(current_year + 1, 1, 1)
                }
            }
        },
        # Group all matched documents and count them
        {
            "$group": {
                "_id": None,
                "total_books_read": {"$count": {}}
            }
        }
    ]

    try:
        # Execute the pipeline
        results = list(db["user_reads"].aggregate(pipeline))
        
        # Extract the total count
        if results:
            total_books_read = results[0]["total_books_read"]
            logger.success(f"A total of {total_books_read} books were read this year.")
        else:
            logger.info("No books were read this year.")
            
    except Exception as e:
        logger.error(f"An error occurred during aggregation: {e}")
    finally:
        client.close()
        logger.info("MongoDB connection closed.")

if __name__ == "__main__":
    count_books_read_this_year()



2025-08-17 00:20:43.924 | INFO     | src.elt.transforms.utils:connect_mongodb:17 - Successfully connected to MongoDB
2025-08-17 00:20:43.958 | SUCCESS  | __main__:count_books_read_this_year:38 - A total of 42 books were read this year.
2025-08-17 00:20:43.980 | INFO     | __main__:count_books_read_this_year:46 - MongoDB connection closed.
